In [7]:
# Add path to other HW folders / Modular Design
import sys
sys.path.append("../")

# Import modules
import numpy as np
import astropy.units as u
from astropy.constants import G
import astropy.table as tbl
from Homework2.ReadFile import Read
from Homework4.CenterOfMass import CenterOfMass

##  Mass Profile Class

In [8]:
class MassProfile:
    """
    :param galaxy: a Galaxy name, e.g. "MW", "M31" or "M33"
    :param snap: snapshot number, e.g. 0, 1, etc
    """
    
    def __init__(self, galaxy, snap):
        """
        
        """
        
        # Reconstruct the filename
        # Format: 'GalaxyName_SnapNum.txt'
        # SnapNum - three digits zero-padding integer
        self.filename= f'{galaxy:s}_{snap:03d}.txt'
        self.gname = galaxy[-3:].upper()
        self.snap = snap
        
    def MassEnclosed(self, ptype, r, delta=0.1):
        """
        compute the mass enclosed within a given radius of the COM 
        position for a specified component of the galaxy.
        
        :param ptype:
        :param r: array of radii  (kpc)
        :param delta: tolerance for COM iterative estimation
        :return:
        """
        # Get CenterOfMass object, and compute its position
        com = CenterOfMass(self.filename, ptype)
        comp = com.COM_P(delta).value
        
        # Make sure input radii is an array, could be a list or scalar
        r_array = np.atleast_1d(r)
        
        # Switch particle coords to COM reference frame
        xNew = com.x - comp[0]
        yNew = com.y - comp[1]
        zNew = com.z - comp[2]
        # Calculate each particle's radial distance to COM
        rNew = np.sqrt(xNew**2 + yNew**2 + zNew**2)
        
        # Initialize array to store enclosed masses; same shape as radii array
        rMasses = np.zeros(r_array.size)
        
        # Loop over each radius
        for i in range(r_array.size):
            # select those within current radius r[i], and sum masses
            rMasses[i] = com.m[rNew < r_array[i]].sum()
        
        return rMasses * 1e10 * u.Msun

    def MassEnclosedTotal(r, delta=0.1):
        ptypes = [1, 2, 3] if self.gname != 'M33' else [1, 2]
        rMasses = np.array([self.MassEnclosed(0, r, delta) for i in ptypes])
        rTotalMasses = rMasses.sum(axis=1)
        return rTotalMasses

    def CircularVelocity(self, ptype, r, delta=0.1):
        Mass = MassEnclosed(ptype, r, delta)
        return np.sqrt(G * Mass / r).to(u.km/u.s)

    def CircularVelocityTotal(r):
        Mass = MassEnclosedTotal(r, delta)
        return np.sqrt(G * Mass / r).to(u.km/u.s)


def HernquistM(r, a=60, Mhalo=1.97):
    """
    Function that returns the Hernquist 1990 mass profile
    
    :param r: Distance from the center of the galaxy (kpc)
    :param a: the scale radius (kpc)
    :param Mhalo: the total dark matter halo mass (10^12 Msun)
    :return: total dark matter halo mass within r (Msun)
    """
    return np.round(Mhalo * r**2 / (a + r)**2, 2) * u.Msun
        
    
def HernquistVCirc(r, a=60, Mhalo=1.97):
    Mass = HernquistM(r, a, Mhalo)
    return np.sqrt(G * Mass / (r * u.kpc)).to(u.km/u.s)


In [9]:
mw_str = '../../SampleData/MW'
m31_str = '../../SampleData/M31'
m33_str = '../../SampleData/M33'

In [10]:
MW = MassProfile(mw_str, 0)

In [11]:
r = np.arange(0.25, 30.5, 1.5)
r

array([ 0.25,  1.75,  3.25,  4.75,  6.25,  7.75,  9.25, 10.75, 12.25,
       13.75, 15.25, 16.75, 18.25, 19.75, 21.25, 22.75, 24.25, 25.75,
       27.25, 28.75, 30.25])

In [12]:
MW.MassEnclosed(1, r, 0.1)

<Quantity [0.00000000e+00, 1.18495500e+09, 4.77931850e+09, 9.12415350e+09,
           1.57599015e+10, 2.36991000e+10, 3.14013075e+10, 4.08809475e+10,
           5.29279900e+10, 6.37110805e+10, 7.59951140e+10, 8.82791475e+10,
           1.02340614e+11, 1.15730605e+11, 1.29871068e+11, 1.44209024e+11,
           1.59376447e+11, 1.73121926e+11, 1.86037935e+11, 1.99269932e+11,
           2.12620425e+11] solMass>